In [6]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

input_size = 784
hidden_size = 500
num_classes = 10

train = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
train_subset, val_subset = torch.utils.data.random_split(train, [50000, 10000], generator=torch.Generator().manual_seed(1))
train_loader = torch.utils.data.DataLoader(dataset=train_subset, shuffle=True, batch_size=100)
val_loader = torch.utils.data.DataLoader(dataset=val_subset, shuffle=True, batch_size=100)
test_loader = torch.utils.data.DataLoader(dataset=test, shuffle=False, batch_size=100)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Conv2d(1, 10, kernel_size=5)
        self.l2 = nn.Conv2d(10, 20, kernel_size=5)
        self.l2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.l1(x), 2))
        x = F.relu(F.max_pool2d(self.l2_drop(self.l2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

nn = Net()
optimizer = optim.Adam(nn.parameters(), lr=0.001)

def train(epoch):
  nn.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = nn(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 200 == 0:
      print("Epoch:", epoch)
      print("Training set accuracy:", ((batch_idx * len(data))/len(train_loader.dataset)*100), "%\t", "Training set loss:", loss.item())

def validate():
    nn.train()
    for batch_idx, (data, target) in enumerate(val_loader):
        optimizer.zero_grad()
        output = nn(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print()
            print("Validation set accuracy:", ((batch_idx * len(data))/len(val_loader.dataset)*100), "%\t", "Validation set loss:", loss.item())
            print()

def test():
  nn.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = nn(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum() 
  print("Test set avg loss:", test_loss, "Accuracy:", correct/len(test_loader.dataset))
  print()

test()
for epoch in range(1, 5 + 1):
  train(epoch)
  validate()
  test()









Test set avg loss: 23081.22833251953 Accuracy: tensor(0.0587)
Epoch: 1
Training set accuracy: 0.0 %	 Training set loss: 2.3169023990631104
Epoch: 1
Training set accuracy: 40.0 %	 Training set loss: 0.7304441928863525
Epoch: 1
Training set accuracy: 80.0 %	 Training set loss: 0.360426664352417
Validation set accuracy: 0.0 %	 Validation set loss: 0.26729241013526917
Test set avg loss: 1339.4778796434402 Accuracy: tensor(0.9584)
Epoch: 2
Training set accuracy: 0.0 %	 Training set loss: 0.3400309383869171
Epoch: 2
Training set accuracy: 40.0 %	 Training set loss: 0.3808347284793854
Epoch: 2
Training set accuracy: 80.0 %	 Training set loss: 0.32410675287246704
Validation set accuracy: 0.0 %	 Validation set loss: 0.23494602739810944
Test set avg loss: 938.762925311923 Accuracy: tensor(0.9704)
Epoch: 3
Training set accuracy: 0.0 %	 Training set loss: 0.21838083863258362
Epoch: 3
Training set accuracy: 40.0 %	 Training set loss: 0.3207089304924011
Epoch: 3
Training set accuracy: 80.0 %	 Traini